In [3]:
#################################### LIBRARY IMPORTS ##########################################
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from scipy.stats import norm
from iminuit import Minuit
from iminuit.cost import UnbinnedNLL
import panel as pn

pn.extension('matplotlib')

#################################### PANEL INPUT WIDGETS ##########################################
n_max_input = pn.widgets.IntInput(name="Max Harmonic (n_max)", value=10, step=1, start=1)
I0_input = pn.widgets.FloatInput(name="Current Magnitude (I0)", value=16000)
#mu0_input = pn.widgets.FloatInput(name="Vacuum Permeability (mu0)", value=4 * np.pi * 1e-7)
ref_rad_input = pn.widgets.FloatInput(name="Reference Radius (ref_rad)", value=1.5)
#r0_input = pn.widgets.FloatInput(name="Start Radius (r0)", value=2.0)
#dr_input = pn.widgets.FloatInput(name="Radial Spacing (dr)", value=0.2)
n_radial_input = pn.widgets.TextInput(name="Radial Wires per Layer (Enter List)", value="[5, 6, 2]")
num_cond_input = pn.widgets.TextInput(name="Cables per Quadrant Enter List", value="[10, 5, 6]")

run_button = pn.widgets.Button(name="Run Plot", button_type="primary")
loading_text = pn.pane.Markdown("")
plot_output = pn.pane.Matplotlib()

#################################### HELPER FUNCTIONS ##########################################

def B_harm(x, y, x_a, y_a, I, mu0, n_max):
    a = np.sqrt(x_a**2 + y_a**2)
    r = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y_a, x_a)
    theta = np.arctan2(y, x)
    ang = phi - theta
    r = np.where(r == 0, 1e-10, r)

    B_r_in = ((mu0 * I) / (2 * np.pi * a)) * np.sum([(r / a)**(n - 1) * np.sin(n * ang) for n in range(1, n_max+1)], axis=0)
    B_theta_in = (-1)*((mu0 * I) / (2 * np.pi * a)) * np.sum([(r / a)**(n - 1) * np.cos(n * ang) for n in range(1, n_max+1)], axis=0)

    B_r_out = ((mu0 * I) / (2 * np.pi * a)) * np.sum([(a / r)**(n + 1) * np.sin(n * ang) for n in range(0, n_max+1)], axis=0)
    B_theta_out = ((mu0 * I) / (2 * np.pi * a)) * np.sum([(a / r)**(n + 1) * np.cos(n * ang) for n in range(0, n_max+1)], axis=0)

    B_r = np.where(r > a, B_r_out, B_r_in)
    B_theta = np.where(r > a, B_theta_out, B_theta_in)

    B_x = B_r * np.cos(theta) - B_theta * np.sin(theta)
    B_y = B_r * np.sin(theta) + B_theta * np.cos(theta)
    return B_x, B_y

def theta_list_maker(u):
    theta_right = np.arcsin(u)
    theta_left = np.pi - theta_right
    theta_bot_left = (-1)*theta_left
    theta_bot_right = (-1)*theta_right
    return np.sort(np.concatenate([theta_right, theta_left, theta_bot_left, theta_bot_right]))

def wire_params(theta_list, radii_list, I0):
    wires = []
    for theta in theta_list:
        for r in radii_list:
            x = r * np.cos(theta)
            y = r * np.sin(theta)
            current = -I0 if x > 0 else (I0 if x < 0 else 0)
            wires.append((x, y, current))
    return wires

#################################### MAIN CALCULATION ##########################################

def run_calculation():
    loading_text.object = "⏳ Loading..."
    
    # Parse values
    n_max = n_max_input.value
    I0 = I0_input.value
    mu0 = mu0_input.value
    ref_rad = ref_rad_input.value
    r0 = r0_input.value
    dr = dr_input.value
    n_radial = eval(n_radial_input.value)
    num_cond_quad = eval(num_cond_input.value)
    
    # Derived constants
    inner_radius = r0 - dr/2
    outer_radius = (r0 + (n_radial[0] - 1) * dr) + dr/2
    num_conductors = 4 * np.array(num_cond_quad)

    # u distribution
    u_list = [np.linspace(0, 1, n + 2)[1:-1] for n in num_cond_quad]
    parent_theta_list = [theta_list_maker(u) for u in u_list]

    # Radial positions
    layer_radii = []
    for j in range(len(n_radial)):
        if j == 0:
            layer_radii.append([r0 + i * dr for i in range(n_radial[0])])
        else:
            start = layer_radii[j-1][-1] + dr
            layer_radii.append([start + i * dr for i in range(n_radial[j])])

    total_wire_params = [wire_params(parent_theta_list[i], layer_radii[i], I0) for i in range(len(layer_radii))]

    # B-field Grid
    bounds = [[-4.8, 4.8], [-4.8, 4.8]]
    x = np.linspace(bounds[0][0], bounds[0][1], 250)
    y = np.linspace(bounds[1][0], bounds[1][1], 250)
    X, Y = np.meshgrid(x, y)
    Bx_total = np.zeros_like(X)
    By_total = np.zeros_like(Y)

    for params in total_wire_params:
        for x0, y0, I in params:
            Bx, By = B_harm(X, Y, x0, y0, I, mu0, n_max)
            Bx_total += Bx
            By_total += By

    B_mag = np.sqrt(Bx_total**2 + By_total**2)

    # Plotting
    fig, ax = plt.subplots(figsize=(8, 8))
    strm = ax.streamplot(X, Y, Bx_total, By_total, color=B_mag, cmap='viridis', density=3, zorder=0)
    cbar = fig.colorbar(strm.lines, ax=ax, label='Magnetic Field Magnitude (T)')

    for params in total_wire_params:
        for x0, y0, I in params:
            color = 'r' if I > 0 else 'b'
            ax.plot(x0, y0, 'o', color=color, zorder=2)

    theta_circle = np.linspace(0, 2*np.pi, 500)
    ax.plot(ref_rad * np.cos(theta_circle), ref_rad * np.sin(theta_circle), 'k:', zorder=1)
    ax.plot(inner_radius * np.cos(theta_circle), inner_radius * np.sin(theta_circle), 'k--', zorder=1)
    ax.plot(outer_radius * np.cos(theta_circle), outer_radius * np.sin(theta_circle), 'k--', zorder=1)

    for i in range(1, len(layer_radii)):
        r_layer = layer_radii[i][-1] + dr/2
        ax.plot(r_layer * np.cos(theta_circle), r_layer * np.sin(theta_circle), 'k--', zorder=1)

    ax.axhline(0, color='black', linewidth=1, zorder=1)
    ax.axvline(0, color='black', linewidth=1, zorder=1)
    ax.set_aspect('equal')
    ax.set_xlim(bounds[0][0], bounds[0][1])
    ax.set_ylim(bounds[1][0], bounds[1][1])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(f"$\\cos \\theta$ Field with harmonics up to $n$ = {n_max}")

    plot_output.object = fig
    loading_text.object = "✅ Done!"

#################################### BUTTON CALLBACK ##########################################
run_button.on_click(lambda event: run_calculation())

#################################### LAYOUT ##########################################

form_inputs = pn.Column(
    n_max_input, I0_input,
    ref_rad_input, n_radial_input, num_cond_input,
    run_button, loading_text,
    width=300
)

layout = pn.Row(form_inputs, plot_output)
layout.servable()

Row
    [0] Column(width=300)
        [0] IntInput(name='Max Harmonic (n_max)', start=1, value=10)
        [1] FloatInput(name='Current Magnitude (I0)', value=16000)
        [2] FloatInput(name='Reference Radius (..., value=1.5)
        [3] TextInput(name='Radial Wires p..., value='[5, 6, 2]')
        [4] TextInput(name='Cables per Quadrant E..., value='[10, 5, 6]')
        [5] Button(button_type='primary', name='Run Plot')
        [6] Markdown(str)
    [1] Matplotlib(None)